# Identificamos skills tecnológicas a través de las descripciones laborales
## Intro
#### Mediante un asistente creado en la [OpenAI developer platform](https://platform.openai.com/) vamos a extraer de las descripciones laborales, la información que queremos con el formato que he indicado al asistente, en este caso un JSON.

#### Esto (mediante entrenamiento) nos ahorra problemas como:
- Tecnologías escritas de diferente forma. (PowerBi, Power BI o Powerbi por ejemplo)
- El idioma de las descripciones.
- Filtros enormes que habría que hacer para el tratamiento de texto (tildes, mayusculas, etc).


### Formato del Json resultante
<img src="../recursos/json_extraido_por_model_ETL.png" height="500px">

---
---

## Importamos librerías y configuraciones

In [1]:
from dotenv import load_dotenv
import os
import openai
from openai import OpenAI
from typing import Optional, Dict, Any, List
import ast
from tqdm import tqdm
import pandas as pd

%load_ext autoreload
%autoreload 2

load_dotenv()

tqdm.pandas()

## creamos diccionario de dataframes con las ofertas

In [2]:
dic_dfs_ofertas = {archivo.rstrip('.csv') : pd.read_csv(f'../datos/datos_recibidos_empresa/{archivo}') for archivo in os.listdir('../datos/datos_recibidos_empresa/') if archivo.endswith('.csv')}

dic_dfs_ofertas.keys()

dict_keys(['ofertas_data_analyst', 'ofertas_data_engineer', 'ofertas_data_scientist', 'ofertas_seguridad_privacy_data'])

## Conectamos con api OpenAI y realizamos la extracion de skills

In [5]:
api_openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# asistente creado y entrenado en el playground de la api
asistente_extractor = api_openai.beta.assistants.retrieve(os.getenv('ASSISTANT_API_KEY'))


def extraer_skills_descripcion(descripcion):

    # por cada ejecución crearé una nueva ejecución ya que aunque es más costoso a nivel de código,
    # es mejor a nivel de tokens ya que no tiene que mantener el contexto de la conversación y para
    # este caso me es mejor.
    conversacion = api_openai.beta.threads.create()

    mensaje = api_openai.beta.threads.messages.create(
    thread_id=conversacion.id,
    role="user",
    content=descripcion
    )

    ejecucion_asistente = api_openai.beta.threads.runs.create_and_poll(
    thread_id=conversacion.id,
    assistant_id=asistente_extractor.id
    )


    if ejecucion_asistente.status == 'completed': 
        mensajes = api_openai.beta.threads.messages.list(
        thread_id=conversacion.id
        )
        
        respuesta = ast.literal_eval(mensajes.data[0].content[0].text.value)
        return respuesta['skills'] if respuesta != None else ['Desconocidas']

    else:
        print(ejecucion_asistente.status)

In [ ]:
for key, value in tqdm(dic_dfs_ofertas.items()):
    print(f'Obteniendo info: {key.upper()}\n')

    df = value
    df['Skills'] = df['Descripcion'].progress_map(lambda x: extraer_skills_descripcion(x))

    df.to_csv('../datos/datos_recibidos_empresa/datos_skills_identificadas/')

In [12]:
dic_dfs_ofertas['ofertas_data_engineer'].sample()

,Titulo,Empresa,Ubicacion,Salario,Experiencia minima,Tipo de contrato,Estudios minimos,Skills,Descripcion,Referencia,cat_original,cat_identificada
244,Data Engineer Junior,BETWEEN,"Zaragoza, España",50.000€ - 70.000€ brutos anuales,1-2 años,"Temporal, media jornada",Máster en Big Data o Ingeniería de Datos,NaN,**Data Engineer Junior en BETWEEN**\n\n ¿Es...,DENG19272,Data Engineer,NaN


100%|██████████| 2/2 [00:12<00:00,  6.00s/it]


In [8]:
df_descripciones_prueba

,descripciones,skills
0,"\nEn CyberSecure Tech, buscamos un/a Data Prot...","[iso 27001, gdpr, auditoria de datos, sistemas..."
1,"\nEn Shield Analytics, buscamos un/a Informati...","[comunicacion efectiva, gestion de riesgos, tr..."
